In [3]:
!pip install 'sagemaker[local]' --upgrade
!pip install -U sagemaker
!pip install -U boto3

In [5]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor
region = sagemaker.Session().boto_region_name
role = get_execution_role()
from sagemaker.processing import ProcessingInput, ProcessingOutput


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Execute evaluation job of summarization task for Claude Model

In [ ]:
ls -l ~/

In [ ]:
!docker images

In [ ]:
!docker rmi 0e95ec168f7b -f


In [ ]:
!docker image prune -f

In [ ]:

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri='XXXXXXXXXXXX.dkr.ecr.us-east-1.amazonaws.com/fmeval-container:latest',
    role=role, 
    base_job_name="fmeval-anthropic-text-summarization",
    instance_type="local", #"ml.m5.xlarge", 
    instance_count=1
)

In [ ]:

script_processor.run(
    code="code/claude_text_summarization.py",
    # arguments = ["arg1", "arg2"], # Arguments can optionally be specified here
    inputs=[ProcessingInput(source="data/xsum_sample.jsonl", 
                            destination="/opt/ml/processing/input/data/"),
           ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output/evaluation")
    ],
    arguments=[#'--region', region,
                '--input_data', 'xsum_sample.jsonl',
               #'--model_id', 'anthropic.claude-v2',
            '--model_id', 'anthropic.claude-v2:1',
               #'--model_id', 'anthropic.claude-3-sonnet-20240229-v1:0',
               #'--model_id', 'anthropic.claude-3-haiku-20240307-v1:0',
               #'--model_id', 'anthropic.claude-instant-v1',
              ],
    wait=True
)

## execute processing job with S3 files

In [ ]:
bucket = sagemaker.Session().default_bucket()
print(bucket)
s3_prefix = "SageMaker_LLM_evaluation_job_package"

In [ ]:
!aws s3 sync code/ "s3://{bucket}/{s3_prefix}/code/"
!aws s3 sync data/ "s3://{bucket}/{s3_prefix}/data/"

In [ ]:
script_processor = ScriptProcessor(
    command=['python3'],
    image_uri='XXXXXXXXXXXX.dkr.ecr.us-east-1.amazonaws.com/fmeval-processing-container:latest',
    role=role, 
    base_job_name="fmeval-text-summarization",
    instance_type="ml.m5.xlarge", 
    instance_count=1
)

In [ ]:
script_processor.run(
    code=f"s3://{bucket}/{s3_prefix}/code/preprocessing.py",
    # arguments = ["arg1", "arg2"], # Arguments can optionally be specified here
    inputs=[ProcessingInput(source= f"s3://{bucket}/{s3_prefix}/data/xsum_sample.jsonl", 
                            destination="/opt/ml/processing/input/data/"),
           ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output/evaluation")
    ],
)

## Execute evaluation job of summarization task for OpenAI model (ChatGPT4)

In [6]:
script_processor = ScriptProcessor(
    command=['python3'],
    image_uri='XXXXXXXXXXXX.dkr.ecr.us-east-1.amazonaws.com/fmeval-container:latest',
    role=role, 
    base_job_name="fmeval-openai-text-summarization",
    instance_type="local", #"ml.m5.xlarge", 
    instance_count=1
)

In [7]:
script_processor.run(
    code="code/openai_text_summarization.py",
    # arguments = ["arg1", "arg2"], # Arguments can optionally be specified here
    inputs=[ProcessingInput(source= f"data/xsum_sample.jsonl", 
                            destination="/opt/ml/processing/input/data/"),
           ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output/evaluation")
    ],
    arguments=['--input_data', 'xsum_sample.jsonl',
               '--model_id', 'gpt-3.5-turbo',
              ],
)

INFO:sagemaker:Creating processing-job with name fmeval-openai-text-summarization-2024-04-11-13-48-11-564
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:docker compose file: 
services:
  sagemaker-local:
    container_name: 3jufvvi4lp-sagemaker-local
    entrypoint:
    - python3
    - /opt/ml/processing/input/code/openai_text_summarization.py
    - --input_data
    - xsum_sample.jsonl
    - --model_id
    - gpt-3.5-turbo
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 820974724107.dkr.ecr.us-east-1.amazonaws.com/fmeval-container:latest
    network_mode: sagemaker
    stdin_open: true
    tty: true
    volumes:
    - /home/sagemaker-user/tmp/tmpqaqh938e/sagemaker-local/output:/opt/ml/output
    - /home/sagemaker-user/tmp/tmpqaqh938e/sagemaker-local/config:/opt/ml/config
    - /home/sagemaker-user/tmp/tmp4_7ab88s:/opt/ml/processing/in

time="2024-04-11T13:48:12Z" level=warning msg="/home/sagemaker-user/tmp/tmpqaqh938e/docker-compose.yaml: `version` is obsolete"
 Container 3jufvvi4lp-sagemaker-local  Creating
 Container 3jufvvi4lp-sagemaker-local  Created
Attaching to 3jufvvi4lp-sagemaker-local
3jufvvi4lp-sagemaker-local  | sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
3jufvvi4lp-sagemaker-local  | sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
3jufvvi4lp-sagemaker-local  | [Errno 17] File exists: '/opt/ml/processing/output/evaluation'
3jufvvi4lp-sagemaker-local  | Could not make directories
3jufvvi4lp-sagemaker-local  | [nltk_data] Downloading package wordnet to /root/nltk_data...
3jufvvi4lp-sagemaker-local  | [nltk_data] Downloading package punkt to /root/nltk_data...
3jufvvi4lp-sagemaker-local  | [nltk_data]   Unzipping tokenizers/punkt.zip.
3jufvvi4lp-sagemaker-local  | [nltk_data] Downloading package omw-1.4 to

RuntimeError: Failed to run: ['docker', 'compose', '-f', '/home/sagemaker-user/tmp/tmpqaqh938e/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit']. Process exited with code: 1